This notebook hijacks the spyking-circus backbone to run a notch filter on your data.

# First, you need to install the packages `mpi4py` and `blosc` in your environment.

`conda install -c conda-forge mpi4py`  
`conda install -c conda-forge python-blosc`  
`conda install -c conda-forge psutil`
  
Note that you might also need to install some other commonly installed packages (e.g. numpy) that aren't listed here.

Note that you might be able to accelerate everything with your GPU on Linux! See the following message that popped up when I installed `mpi4py`:  

```
For Linux 64, Open MPI is built with CUDA awareness but this support is disabled by default.
To enable it, please set the environment variable OMPI_MCA_opal_cuda_support=true before
launching your MPI processes. Equivalently, you can set the MCA parameter in the command line:
mpiexec --mca opal_cuda_support 1 ...
 
In addition, the UCX support is also built but disabled by default.
To enable it, first install UCX (conda install -c conda-forge ucx). Then, set the environment
variables OMPI_MCA_pml="ucx" OMPI_MCA_osc="ucx" before launching your MPI processes.
Equivalently, you can set the MCA parameters in the command line:
mpiexec --mca pml ucx --mca osc ucx ...
Note that you might also need to set UCX_MEMTYPE_CACHE=n for CUDA awareness via UCX.
Please consult UCX's documentation for detail.
```


Just like for spyking-circus, you need to have a `session_name.dat` and `session_name.params` file in the same folder before running.  
  
~~At this point, you want to have the `overwrite` flag set to `False`, unlike for normal running of spyking-circus.~~  
NVM - seems to be lots of issues in spyking-circus with this.

In [3]:
import sys
sys.path.append('/data/GitHub/spyking-circus/')

from circus.shared.parser import CircusParser
from circus.filtering import main

def miniscope_filter(dir_use: str, dat_file_name: str ='continuous_combined.dat', top_limit: int or float = 540):
    """Filters file to get rid of EWL + 60Hz + harmonic noise"""
    
    dir_use = Path('/data/Working/Trace_FC/Recording_Rats/Finn/2022_01_18_habituation')
    dat_file = sorted(dir_use.glob("*" + dat_file_name))[0]
    params = CircusParser(str(dat_file))
    # Set up filter for EWL, EWL harmonic and 60Hz
    notch_filter = [{'w0': 4843.0, 'bw': 45, 'Q': None}, {'w0': 4843.0*3, 'bw': 35, 'Q': None}, 
                    {'w0': 60.0, 'bw': None, 'Q': 30.0}] 
    
    # Set up filters for 60Hz harmonics up to top_limit - might want to lower top limit to bottom limit of bandpass filter in spyking-circus
    harmonics = np.arange(180, top_limit+1, 120)
    bw_harm = 4
    for harmonic in harmonics:
        notch_filter.append({'w0': harmonic, 'bw': bw_harm, 'Q': None})

    for idf, filter in enumerate(notch_filter):
        main(params, 8, 0, False, filter, idf == len(notch_filter) - 1)
